RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

https://stackoverflow.com/questions/48274929/pytorch-runtimeerror-trying-to-backward-through-the-graph-a-second-time-but

In [48]:
import torch
import torch.distributions as D
import torch.nn as nn

In [55]:
class Model(nn.Module):

    def __init__(self):
        super(Model, self).__init__()
        random_uniform_val = torch.rand(2)
        # 1 bernouli ==> 2 classification with 1 feature
        self.covariance = nn.Parameter(torch.full((2,1),1/2))
        self.mean = nn.Parameter(random_uniform_val/random_uniform_val.sum())
        # print(self.covariance)
        # print(self.mean)
        self.mix_model = D.MixtureSameFamily(D.Categorical(self.mean),
                                        D.Independent(D.Bernoulli(self.covariance), reinterpreted_batch_ndims=1))
    
    def forward(self,data):
        log_prob = self.mix_model.log_prob(data)
        return -sum(log_prob)/len(data)
    
model = Model()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
running_loss = 0.
training_data = torch.asarray([[[[1.]]*20 + [[0.]]*80],[[[1.]]*30 + [[0.]]*70],[[[1.]]*40 + [[0.]]*60],[[[1.]]*50 + [[0.]]*50]])
for i in range(4):
    # training_data = torch.torch.randint(1,[45,1]).float()
    optimizer.zero_grad()
    loss = model(training_data[i])
    # print(loss)
    optimizer.step()
    running_loss += loss.item()
print(running_loss)
# training_data = torch.asarray([[1.]]*20 + [[0.]]*80)
# result = model(training_data)
# print(result)

2.772590160369873


In [90]:
import math
prob = 0.5
ratio = [7,3]
NLL = -(ratio[0]*math.log(prob)+ratio[1]*math.log(1-prob))
print(math.log(prob))

-0.6931471805599453
